In [11]:
import pandas as pd
import numpy as np
from wordcloud import WordCloud
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
lemmatizer = nltk.stem.WordNetLemmatizer()
w_tokenizer = TweetTokenizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Archana\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Archana\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
%%time
df1 = pd.read_excel('Social_Listening_Excel_SQL_JAN17_AUG20.xlsx')
df2 = pd.read_excel('SOCIAL_LISTENING_EXCEL_SQL_SEP`20_MAY`21.xlsx')
print(df1.shape)
print(df2.shape)

(499743, 24)
(161831, 24)
Wall time: 2min 49s


In [3]:
print(df1[df1['Content'].isna()].shape)
print(df2[df2['Content'].isna()].shape)

(1390, 24)
(142, 24)


In [4]:
df1.head()

,Brand,Year,Month,Date,URL,Domain,Platform,Sentiment,Content,Category_1,...,User_Name,Author_Name,Author_Type,Followers,Profile_Details,Profile_Link,Location,MentionID,Language,File
0,Scorpio,2017,Jan'17,42766.0,https://www.carwale.com/news//news/new-mahindr...,NaN,Mainstream Media,Neutral,"16 hours ago - First introduced in 2007, the M...",Classified,...,NaN,NaN,-,0,NaN,NaN,NaN,NaN,NaN,Locobuzz
1,Scorpio,2017,Jan'17,42766.0,https://www.carwale.com/used/cars-in-hyderabad...,NaN,Mainstream Media,Neutral,1 day ago - CarWale - Used Mahindra Scorpio [2...,Classified,...,NaN,NaN,-,0,NaN,NaN,NaN,NaN,NaN,Locobuzz
2,Scorpio,2017,Jan'17,42765.0,https://www.carwale.com/used/cars-in-hyderabad...,NaN,Mainstream Media,Neutral,20 hours ago - CarWale - Used Mahindra Scorpio...,Classified,...,NaN,NaN,-,0,NaN,NaN,NaN,NaN,NaN,Locobuzz
3,Scorpio,2017,Jan'17,42766.0,http://indianautosblog.com/2017/01/2017-mahind...,NaN,Mainstream Media,Neutral,1 hour ago - The 2017 Mahindra Scorpio Getaway...,Classified,...,NaN,NaN,-,0,NaN,NaN,NaN,NaN,NaN,Locobuzz
4,Scorpio,2017,Jan'17,42765.0,https://www.carwale.com/used/cars-in-kolkata/m...,NaN,Mainstream Media,Neutral,19 hours ago - CarWale - Used Mahindra Scorpio...,Classified,...,NaN,NaN,-,0,NaN,NaN,NaN,NaN,NaN,Locobuzz


In [5]:
df1.columns

Index(['Brand', 'Year', 'Month', 'Date', 'URL', 'Domain', 'Platform',
       'Sentiment', 'Content', 'Category_1', 'Category_2', 'Catgeory_3',
       'Category_4', 'Category_5', 'User_Name', 'Author_Name', 'Author_Type',
       'Followers', 'Profile_Details', 'Profile_Link', 'Location', 'MentionID',
       'Language', 'File'],
      dtype='object')

In [6]:
df2.columns

Index(['Brand', 'Year', 'Month', 'Date', 'URL', 'Domain', 'Platform',
       'Sentiment', 'Content', 'Category_1', 'Category_2', 'Catgeory_3',
       'Category_4', 'Category_5', 'User_Name', 'Author_Name', 'Author_Type',
       'Followers', 'Profile_Details', 'Profile_Link', 'Location', 'MentionID',
       'Language', 'File_1'],
      dtype='object')

In [7]:
df = pd.concat([df1[['Brand', 'Sentiment', 'Content']], df1[['Brand', 'Sentiment', 'Content']]], ignore_index=True)
df.shape

(999486, 3)

In [8]:
df.head()

,Brand,Sentiment,Content
0,Scorpio,Neutral,"16 hours ago - First introduced in 2007, the M..."
1,Scorpio,Neutral,1 day ago - CarWale - Used Mahindra Scorpio [2...
2,Scorpio,Neutral,20 hours ago - CarWale - Used Mahindra Scorpio...
3,Scorpio,Neutral,1 hour ago - The 2017 Mahindra Scorpio Getaway...
4,Scorpio,Neutral,19 hours ago - CarWale - Used Mahindra Scorpio...


In [9]:
# checking the content length related stats

df['Content'].apply(lambda x: len(x.split()) if type(x) == str else 0).describe()

count    999486.000000
mean         26.748335
std          49.107054
min           0.000000
25%           6.000000
50%          19.000000
75%          32.000000
max        4540.000000
Name: Content, dtype: float64

In [12]:
def lemmatize_text(text):
    return [(lemmatizer.lemmatize(w)) for w in w_tokenizer.tokenize((text))]

def remove_punctuation(words):
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', (word))
    if new_word != '':
       new_words.append(new_word)
    return new_words

In [ ]:
def preprocess_text(text):
    # remove hyperlinks
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
    # it will remove hashtags. We have to be careful here not to remove 
    # the whole hashtag because text of hashtags contains huge information. 
    # only removing the hash # sign from the word
    text = re.sub(r'#', '', text)
    # it will remove single numeric terms in the text. 
    text = re.sub(r'[0-9]', '', text)
    # remove 
    text = text.str.lower()
    text = re.sub(r'day ago - |hours ago - |hour ago|CarWale', '', text)
    return text